# **SnP Market Pre-processing**

In [1]:
#SnP 데이터
import pandas as pd

SnP500 = pd.read_csv('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/SnP500구성종목_wikipedia.csv')

columns_to_read = ['pricingDate', 'tickerSymbol', 'ISOCode', 'priceClose', 'divAdjFactor']
market1 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market1.parquet', columns=columns_to_read)
market2 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market2.parquet', columns=columns_to_read)
market3 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market3.parquet', columns=columns_to_read)
market4 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market4.parquet', columns=columns_to_read)

In [ ]:
# 'ISOCode' 열에 있는 고유한 값들을 확인
unique_iso_codes = market1['ISOCode'].unique()

# 결과 출력
print(unique_iso_codes)

In [ ]:
# 'ISOCode'가 'USD'인 행들만 유지
market1_usd = market1[market1['ISOCode'] == 'USD']
market2_usd = market2[market2['ISOCode'] == 'USD'] 
market3_usd = market3[market3['ISOCode'] == 'USD'] 
market4_usd = market4[market4['ISOCode'] == 'USD']

In [ ]:
#SnP 구성 종목들만 필터링
snP500_symbols = SnP500['Symbol'].unique()
market1_filtered = market1_usd[market1_usd['tickerSymbol'].isin(snP500_symbols)]
market2_filtered = market2_usd[market2_usd['tickerSymbol'].isin(snP500_symbols)]
market3_filtered = market3_usd[market3_usd['tickerSymbol'].isin(snP500_symbols)]
market4_filtered = market4_usd[market4_usd['tickerSymbol'].isin(snP500_symbols)]

#데이터프레임 병합
merged_market_data = pd.concat([market1_filtered, market2_filtered, market3_filtered, market4_filtered])

In [ ]:
#결측치 확인(없음)
missing_price_data = merged_market_data[merged_market_data['priceClose'].isna()]

print(missing_price_data)

In [ ]:
# 수정종가 열 추가
# divadjfactor가 nan값인 경우 수정종가 계산이 안되는 문제가 있음. 이를 해결하기 위해 divadjfactor가 nan값인 경우 1로 대체
merged_market_data['divAdjFactor'].fillna(1, inplace=True)
merged_market_data['adjpriceClose'] = merged_market_data['priceClose'] * merged_market_data['divAdjFactor']
# 필요없는 열 삭제
merged_market_data = merged_market_data.drop(columns=['priceClose', 'divAdjFactor'])

In [ ]:
merged_market_data.to_csv('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/merged_market_data.csv', index=False)

In [ ]:
# merged_market_data 종목 수
unique_ticker_symbols = merged_market_data['tickerSymbol'].nunique()
unique_ticker_symbols

In [ ]:
# merged_market_data 재정렬 후 기간 맞춤

a = merged_market_data.pivot_table(index='pricingDate', columns='tickerSymbol', values='adjpriceClose' ,aggfunc='first')
a = a['2018-05-01':'2023-10-31']
a

In [ ]:
# 각 column별 nan값 확인
# nan값은 평균 178개를 고정적으로 가지고 있는 column이 많음. (480종목)

a.isnull().sum()

In [ ]:
#nan값이 178이상인 종목들을 drop

nan_counts = a.isna().sum()

columns_to_drop = nan_counts[nan_counts > 178].index.tolist()

a_filtered = a.drop(columns=columns_to_drop)

a_filtered

In [ ]:
# NaN값이 178개 이상인 종목들을 drop한 후 저장
# 일단 최종적으로 SnP500 지수는 총 485종목을 볼 것임
# 5년의 기간에서 178일을 제거

a_filtered = a_filtered.dropna(axis=0)
a_filtered.to_csv('/Users/jaekyunpark/Documents/GitHub/S-PQuantTeam01/new_market.csv')
a_filtered